In [4]:
import pandas as pd
import glob
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
def load_asd_data_with_pandas(data_path: str = "/content/drive/MyDrive/Antibody_Chekalin_Denis/asd") -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        # print(f"Загружаем файл: {os.path.basename(file_path)}")
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


### Распределение типов антител

In [11]:
# Создаем категории записей
def categorize_antibody(row):
    """Категоризирует запись по типу антитела"""
    has_heavy = pd.notna(row['heavy_sequence']) and row['heavy_sequence'] != ''
    has_light = pd.notna(row['light_sequence']) and row['light_sequence'] != ''
    
    if row['scfv'] == True:
        return 'scFv'
    elif row['nanobody'] == True:
        return 'Nanobody'
    elif has_heavy and has_light:
        return 'Paired (heavy + light)'
    elif has_heavy and not has_light:
        return 'Heavy-only'
    elif not has_heavy and has_light:
        return 'Light-only'
    else:
        return 'No sequences'

# Применяем категоризацию
agab_df['antibody_type'] = agab_df.apply(categorize_antibody, axis=1)

# Распределение по типам
print("Распределение записей по типам антител:")
print(agab_df['antibody_type'].value_counts())
print("\nПроцентное распределение:")
print(agab_df['antibody_type'].value_counts(normalize=True) * 100)

# Фокус на записях с scFv == False и nanobody == False
print("\n" + "="*60)
print("Детальный анализ записей с scFv == False и nanobody == False:")
print("="*60)

filtered_df = agab_df[
    (agab_df['scfv'] == False) 
    & (agab_df['nanobody'] == False)
]

print(f"\nВсего таких записей: {len(filtered_df)}")

# Распределение по типам среди отфильтрованных
print("\nРаспределение по типам среди отфильтрованных записей:")
filtered_types = filtered_df.apply(categorize_antibody, axis=1)
print(filtered_types.value_counts())
print("\nПроцентное распределение:")
print(filtered_types.value_counts(normalize=True) * 100)


Распределение записей по типам антител:
antibody_type
Paired (heavy + light)    716176
Heavy-only                188742
scFv                      132157
Nanobody                  110904
Light-only                 79104
Name: count, dtype: int64

Процентное распределение:
antibody_type
Paired (heavy + light)    58.364104
Heavy-only                15.381356
scFv                      10.770013
Nanobody                   9.038019
Light-only                 6.446508
Name: proportion, dtype: float64

Детальный анализ записей с scFv == False и nanobody == False:

Всего таких записей: 980878

Распределение по типам среди отфильтрованных записей:
Paired (heavy + light)    713227
Heavy-only                188626
Light-only                 79025
Name: count, dtype: int64

Процентное распределение:
Paired (heavy + light)    72.713120
Heavy-only                19.230322
Light-only                 8.056557
Name: proportion, dtype: float64


In [13]:
print(f"Всего записей с scFv == False и nanobody == False: {len(filtered_df)}")

# Смотрим на распределение по dataset
print("\nРаспределение по dataset для записей с пустыми light_sequence:")
print(empty_light['dataset'].value_counts().head(10))

print("\nРаспределение по dataset для записей с пустыми heavy_sequence:")
print(empty_heavy['dataset'].value_counts().head(10))


Всего записей с scFv == False и nanobody == False: 980878

Распределение по dataset для записей с пустыми light_sequence:
dataset
patents       76588
covid-19      54625
hiv           48008
met            4000
biomap         2725
genbank        2603
literature       42
aae              35
Name: count, dtype: int64

Распределение по dataset для записей с пустыми heavy_sequence:
dataset
patents    78737
genbank      288
Name: count, dtype: int64


### Смотрим на примере dataset = patents

In [16]:
# Считаем количество записей с пустыми light_sequence и heavy_sequence для dataset = patents

# Фильтрация по dataset = patents и scFv == False и nanobody == False
patents_df = agab_df[
    (agab_df['dataset'] == 'patents')
    & (agab_df['scfv'] == False)
    & (agab_df['nanobody'] == False)
]

# Условия для пустых light_sequence и heavy_sequence
empty_light_patents = patents_df['light_sequence'].isnull() | (patents_df['light_sequence'] == '')
empty_heavy_patents = patents_df['heavy_sequence'].isnull() | (patents_df['heavy_sequence'] == '')

# Подсчёт количества каждого случая
both_empty = patents_df[empty_light_patents & empty_heavy_patents].shape[0]
only_light_empty = patents_df[empty_light_patents & ~empty_heavy_patents].shape[0]
only_heavy_empty = patents_df[~empty_light_patents & empty_heavy_patents].shape[0]
none_empty = patents_df[~empty_light_patents & ~empty_heavy_patents].shape[0]

print("Для dataset = patents:")
print(f"  Пустые оба (light_sequence и heavy_sequence): {both_empty}")
print(f"  Пустой только light_sequence: {only_light_empty}")
print(f"  Пустой только heavy_sequence: {only_heavy_empty}")
print(f"  Оба заполнены: {none_empty}")


Для dataset = patents:
  Пустые оба (light_sequence и heavy_sequence): 0
  Пустой только light_sequence: 76588
  Пустой только heavy_sequence: 78737
  Оба заполнены: 21621
